This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_uploads/pytorch-upload-tutorials).

## Wallaroo Model Upload via the Wallaroo SDK: Pytorch Multiple Input Output

The following tutorial demonstrates how to upload a Pytorch Multiple Input Output model to a Wallaroo instance.

### Tutorial Goals

Demonstrate the following:

* Upload a Pytorch Multiple Input Output to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* A Wallaroo version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd


### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [2]:

wl = wallaroo.Client()

### Set Variables and Helper Functions

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

We'll set up some helper functions that will either use existing workspaces and pipelines, or create them if they do not already exist.

In [3]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline


import string
import random

# make a random 4 character suffix to prevent overwriting other user's workspaces
suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

suffix='jch'

workspace_name = f'pytorch-multi-io{suffix}'
pipeline_name = f'pytorch-multi-io'

model_name = 'pytorch-multi-io'
model_file_name = "./models/model-auto-conversion_pytorch_multi_io_model.pt"

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = get_workspace(workspace_name)
wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Configure Data Schemas

The following parameters are required for PyTorch models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a PyTorch model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Upload Method Optional, PyTorch model Required*) | Set as the `Framework.PyTorch`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, PyTorch model Required*) | The input schema in Apache Arrow schema format. Note that float values **must** be `pyarrow.float32()`. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, PyTorch model Required*) | The output schema in Apache Arrow schema format. Note that float values **must** be `pyarrow.float32()`. |
| `convert_wait` | `bool` (*Upload Method Optional, PyTorch model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

In [33]:
input_schema = pa.schema([
    pa.field('input_1', pa.list_(pa.float32(), list_size=10)),
    pa.field('input_2', pa.list_(pa.float32(), list_size=5))
])
output_schema = pa.schema([
    pa.field('output_1', pa.list_(pa.float32(), list_size=3)),
    pa.field('output_2', pa.list_(pa.float32(), list_size=2))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.PYTORCH`.

In [34]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.PYTORCH, 
                        input_schema=input_schema, 
                        output_schema=output_schema
                       )
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.
Model is attempting loading to a native runtime.successful

Ready


Name,pytorch-multi-io
Version,3ef186ed-1e2b-4699-a37a-5b05e0287d65
File Name,model-auto-conversion_pytorch_multi_io_model.pt
SHA,792db9ee9f41aded3c1d4705f50ccdedd21cafb8b6232c03e4a849b6da1050a8
Status,ready
Image Path,None
Architecture,None
Updated At,2023-19-Oct 22:17:36


In [35]:
model.config().runtime()

'onnx'

### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [36]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [38]:
# clear the pipeline if it was used before
pipeline.clear()

pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.3.128',
   'name': 'engine-77c4d6dbb4-nthvl',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'pytorch-multi-io',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'pytorch-multi-io',
      'version': '3ef186ed-1e2b-4699-a37a-5b05e0287d65',
      'sha': '792db9ee9f41aded3c1d4705f50ccdedd21cafb8b6232c03e4a849b6da1050a8',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.2.187',
   'name': 'engine-lb-584f54c899-mbq8z',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [39]:
mock_inference_data = [np.random.rand(10, 10), np.random.rand(10, 5)]
mock_dataframe = pd.DataFrame(
    {
        "input_1": mock_inference_data[0].tolist(),
        "input_2": mock_inference_data[1].tolist(),
    }
)

In [40]:
pipeline.infer(mock_dataframe)

,time,in.input_1,in.input_2,out.output_1,out.output_2,check_failures
0,2023-10-19 22:19:14.125,"[0.4327160941, 0.6680523081, 0.5832859885, 0.0...","[0.2347998092, 0.0456354817, 0.4508901177, 5.2...","[-0.026637405, 0.05912261, 0.16849947]","[0.14986634, 0.0044104904]",0
1,2023-10-19 22:19:14.125,"[0.2590309749, 0.3577258841, 0.2518631052, 0.3...","[0.5622961836, 0.2350844684, 0.2017416527, 0.0...","[-0.030408666, -0.07442912, 0.1161211]","[0.029797293, 0.034324855]",0
2,2023-10-19 22:19:14.125,"[0.9696389168, 0.4615495636, 0.6081982391, 0.6...","[0.0052452519, 0.526622605, 0.8995469378, 0.68...","[-0.11566387, -0.045465104, 0.27442083]","[-0.005684576, -0.0052274913]",0
3,2023-10-19 22:19:14.125,"[0.6619223108, 0.1478579154, 0.1367208066, 0.4...","[0.3269275107, 0.6041623132, 0.2876937081, 0.6...","[-0.049019977, -0.09353256, 0.22203638]","[0.08988738, -0.17687897]",0
4,2023-10-19 22:19:14.125,"[0.6674092156, 0.9862123394, 0.0949382095, 0.0...","[0.5379806839, 0.4730633663, 0.9061706484, 0.8...","[-0.04075108, -0.05331666, 0.34851304]","[0.076819465, -0.027034491]",0
5,2023-10-19 22:19:14.125,"[0.1637691263, 0.0792595736, 0.3310374199, 0.3...","[0.9930933779, 0.8937510858, 0.0840548915, 0.1...","[-0.030322284, 0.009279203, 0.03645277]","[0.009268863, -0.13072528]",0
6,2023-10-19 22:19:14.125,"[0.2676971724, 0.7152151127, 0.8628973481, 0.3...","[0.4350560182, 0.0709558087, 0.7948290474, 0.4...","[-0.013946526, 0.07283753, 0.36205345]","[0.10368031, 0.09944552]",0
7,2023-10-19 22:19:14.125,"[0.1989475509, 0.2401542202, 0.8126796473, 0.1...","[0.2739875725, 0.0324299554, 0.2523070537, 0.4...","[-0.07782371, 0.0528352, 0.212226]","[0.18431139, 0.009309709]",0
8,2023-10-19 22:19:14.125,"[0.5374385017, 0.8880117144, 0.359969765, 0.66...","[0.3012888467, 0.6528897585, 0.8792932783, 0.8...","[-0.122693785, -0.03799792, 0.2594204]","[0.10170637, 0.019279242]",0
9,2023-10-19 22:19:14.125,"[0.7879625177, 0.7247772362, 0.2257518562, 0.3...","[0.2286801827, 0.9383274992, 0.0652973987, 0.0...","[-0.08041412, 0.053865973, 0.07040782]","[0.2569228, -0.16891222]",0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [41]:
pipeline.undeploy()

name,pytorch-multi-io
created,2023-10-19 21:47:11.881873+00:00
last_updated,2023-10-19 22:18:54.037677+00:00
deployed,False
tags,
versions,"39168af2-bd41-49f8-8f5e-b1aba608ee68, d543d8ac-ba93-4dd5-a8d4-8bd2b405eb18, c55a8525-f108-404a-b0da-2f78f2ea2e34, c0358f0b-22b1-431f-8eee-73eef2ce8bbc"
steps,pytorch-multi-io
published,False
